# User Groups

In [1]:
# import required packages
import pandas as pd
from collections import defaultdict
from tqdm import tqdm

In [2]:
# config
ds_name = 'Gowalla'

In [12]:
# loading the Book-Crossing dataset
dataset = pd.read_csv(f"datasets/{ds_name}/{ds_name}_checkins.txt", sep="\t", names=['uid', 'lid', 'ctime'])

In [14]:
dataset.head()

,uid,lid,ctime
0,0,2049,1.287504e+09
1,0,0,1.287411e+09
2,0,1,1.287330e+09
3,0,328,1.287315e+09
4,0,14774,1.287226e+09


In [16]:
# get user distribution
user_dist = dataset['uid'].value_counts()
num_users = len(user_dist)
print('No. users: ' + str(num_users))
print('Mean POIS per user: ' + str(user_dist.mean()))
print('Min POIs per user: ' + str(user_dist.min()))
print('Max POIs per user: ' + str(user_dist.max()))

No. users: 5628
Mean POIS per user: 109.9184434968017
Min POIs per user: 14
Max POIs per user: 1622


In [17]:
# get item distribution
item_dist = dataset['lid'].value_counts()
num_items = len(item_dist)
print('No. items: ' + str(num_items))

No. items: 30943


In [18]:
# get top items
top_fraction = 0.2
num_top = int(top_fraction * num_items)
top_item_dist = item_dist[:num_top]
print('No. top items: ' + str(num_top))

No. top items: 6188


In [19]:
top_item_set = set(top_item_dist.keys())

In [20]:
user_books = defaultdict(set)
for eachline in tqdm(dataset.iterrows()):
    uid, bid, rating = eachline[1][0], eachline[1][1], eachline[1][2] 
    uid, bid = int(uid), int(bid)
    if uid in user_books.keys():
        user_books[uid].add(bid)
    else:
        user_books[uid] = {bid}

618621it [01:06, 9254.07it/s]


In [21]:
user_pop_book_ratio = {}

for user, books in tqdm(user_books.items()):
    if user not in user_pop_book_ratio.keys():
        user_pop_book_ratio[user] = (len(set(books) & set(top_item_set))) / len(set(books))

100%|██████████| 5628/5628 [00:00<00:00, 5846.40it/s]


In [22]:
sorted_user_pop_book_ratio = {k: v for k, v in sorted(user_pop_book_ratio.items(), key=lambda item: item[1], reverse=True)}

In [23]:
num_user = len(sorted_user_pop_book_ratio)
num_top_users = int(top_fraction * num_users)
num_top_users

1125

In [32]:
# Files to save users ids based on the number of checkins
inactive_users = open(f'datasets/{ds_name}/user-groups/inactive_users.txt', 'w')
inactive_users.write('user_id' + '\n')

medium_users = open(f'datasets/{ds_name}/user-groups/medium_users.txt', 'w')
medium_users.write('user_id' + '\n')

active_users = open(f'datasets/{ds_name}/user-groups/active_users.txt', 'w')
active_users.write('user_id' + '\n')

8

In [33]:
for uid in list(sorted_user_pop_book_ratio.keys())[:num_top_users]:
    active_users.write(str(uid) + '\n')
active_users.close()

In [27]:
# for uid in list(sorted_user_pop_book_ratio.keys())[num_top_users:len(sorted_user_pop_book_ratio) - num_top_users]:
#     medium_users.write(str(uid) + '\n')
# medium_users.close()

In [ ]:
for uid in list(sorted_user_pop_book_ratio.keys())[num_top_users:]:
    inactive_users.write(str(uid) + '\n')
inactive_users.close()

In [28]:
for uid in list(sorted_user_pop_book_ratio.keys())[len(sorted_user_pop_book_ratio) - num_top_users:len(sorted_user_pop_book_ratio)]:
    inactive_users.write(str(uid) + '\n')
inactive_users.close()

In [29]:
# file closing
inactive_users.close()
medium_users.close()
active_users.close()